## Setup Dataset

In [ ]:
from src.data.setup import download_and_extract

download_and_extract(
    url="https://drive.google.com/uc?id=1VAc_MFUAlrtqZanxwYRRtGYVLFunKABf",
    zip_path="data.zip",
    extract_dir="data",
    verbose=True,
    remove_zip=True
)

## Offline-Train All Classifiers

In [ ]:
from src.models.decision_tree.train import main as train_decision_tree
from src.models.hoeffding.train import main as train_hoeffding_tree
from src.models.weighted_forest.train import main as train_weighted_forest

# train_decision_tree()
# train_hoeffding_tree()
train_weighted_forest()

## Launch Interactive Game

Left player controls the paddle using Q (up) and A (down).

Choose the enemy using the mode variable. Choices include:
- "human": play against another player
- "pc": pc player aiming to reach ball_y
- "dt": offline-only trained decision tree
- "ht": offline-only pre-trained hoeffding tree
- "wf:" offline-only pre-trained weighted forest

In [ ]:
from src.main import main

mode = "wf" # "human", "pc", "dt", "ht", "wf"
main(mode)

## Evaluate All Offline Pre-Trained Models

Left paddle will be controlled by PC player.

Evaluation metric is the amount of ball collisions by the tested model and final game score after max-score was reached by one player.

In [ ]:
from src.evaluation import evaluate_all_models

results = evaluate_all_models(max_score=3)

for model_name, result in results.items():
    print(f"{model_name.upper()}: {result.survival_time_seconds:.2f}s, "
          f"Score {result.final_pc_score}-{result.final_ai_score}, "
          f"Hits: {result.total_hits}")

## Online Training - Hoeffding Tree

In [ ]:
from src.training.train_online import train_hoeffding_online
from src.training.train_online import train_weighted_forest_online

train_hoeffding_online(
    pretrained_model_path="models/ht/hoeffding_tree_pong.pkl",
    num_episodes=2,
    max_score_per_episode=3,
    save_interval=5
)

# train_weighted_forest_online(
#     pretrained_model_path="models/wf/weighted_forest_pong.pkl",
#     metadata_path="models/wf/weighted_forest_metadata.pkl",
#     num_episodes=10,
#     max_score_per_episode=3,
#     save_interval=5
# )

## Compare Pretrained vs Online Models

In [ ]:
from src.evaluation import compare_pretrained_vs_online

results = compare_pretrained_vs_online(max_score=5)

## Visualize Online Training Metrics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ht_metrics = pd.read_csv("models/ht/hoeffding_online_metrics.csv")
wf_metrics = pd.read_csv("models/wf/weighted_forest_online_metrics.csv")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(ht_metrics['episode'], ht_metrics['survival_seconds'], marker='o')
axes[0, 0].set_title('Hoeffding Tree - Survival Time')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Survival (seconds)')

axes[0, 1].plot(ht_metrics['episode'], ht_metrics['progressive_accuracy'], marker='o')
axes[0, 1].set_title('Hoeffding Tree - Progressive Accuracy')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Accuracy')

axes[1, 0].plot(wf_metrics['episode'], wf_metrics['survival_seconds'], marker='o')
axes[1, 0].set_title('Weighted Forest - Survival Time')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Survival (seconds)')

axes[1, 1].plot(wf_metrics['episode'], wf_metrics['num_cells'], marker='o')
axes[1, 1].set_title('Weighted Forest - Active Cells')
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Number of Cells')

plt.tight_layout()
plt.show()

## Statistical Analysis (5x2 CV)

In [ ]:
from src.statistical_tests import run_rskf, print_results
from src.data.loader import load_training_data
from src.data.preparation import min_max_scale, convert_str_to_int, undersample
from sklearn.tree import DecisionTreeClassifier
from river.tree import HoeffdingTreeClassifier
from src.models.weighted_forest.clf import WeightedForest, euclidean_distance
from sklearn.metrics import accuracy_score
import numpy as np

X, y = load_training_data(random_state=42)
X_np = X.to_numpy()
y_np = y.to_numpy()
X_np, _, _ = min_max_scale(X_np)
y_np, class_mapping = convert_str_to_int(y_np)
X_np, y_np = undersample(X_np, y_np, random_seed=42)

def train_eval_fn(model_name, X_train, y_train, X_test, y_test):
    if model_name == 'DecisionTree':
        clf = DecisionTreeClassifier(max_depth=20, min_samples_split=10, random_state=42)
        clf.fit(X_train, y_train)
        return accuracy_score(y_test, clf.predict(X_test))
    elif model_name == 'HoeffdingTree':
        clf = HoeffdingTreeClassifier()
        for x, y_label in zip(X_train, y_train):
            clf.learn_one(dict(enumerate(x)), y_label)
        preds = [clf.predict_one(dict(enumerate(x))) for x in X_test]
        return accuracy_score(y_test, preds)
    elif model_name == 'WeightedForest':
        clf = WeightedForest(X_train.shape[1], len(np.unique(y_train)), euclidean_distance,
                             accuracy_goal=0.65, random_seed=42)
        clf.fit(X_train, y_train, epochs=3)
        return accuracy_score(y_test, clf.predict(X_test).astype(int))

models = {'DecisionTree': 'DecisionTree', 'HoeffdingTree': 'HoeffdingTree', 'WeightedForest': 'WeightedForest'}

output = run_rskf(train_eval_fn, models, X_np, y_np, n_repeats=10, n_splits=5, random_state=42)
print_results(output)